In [15]:
import requests
import json
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./data/최종_최종__월별기상관측.csv', encoding='cp949')
df = df.drop(['Unnamed: 0'], axis=1)

In [16]:
df = pd.read_csv('./data/최종_최종__월별기상관측.csv', encoding='cp949')
df = df.drop(['Unnamed: 0'], axis=1)

In [17]:
region = df['행정구역'].unique()
region

array(['강원도', '경기도', '인천광역시', '서울특별시', '경상북도', '충청북도', '충청남도', '대전광역시',
       '전라남도', '대구광역시', '전라북도', '울산광역시', '경상남도', '광주광역시', '부산광역시', '제주도',
       '세종특별자치시'], dtype=object)

In [18]:
# g = df[(df['행정구역'] == '강원도')]

In [19]:
# g

In [20]:
# t_date = g['일시'].unique()

In [21]:
# t_date

In [22]:
#t = g[(g['일시'] == '2000-01')]

In [23]:
#t

In [24]:
#t.describe()['평균기온(°C)']

In [26]:
#t.describe()['평균기온(°C)']['50%']

In [27]:
def calc_median(df:pd.DataFrame, colname:str):
    # df(데이터 프레임) colname의 중앙값 산출하는 함수
    return df.describe()[colname]['50%']

In [30]:
reg_df = pd.DataFrame()

for reg in tqdm(region):
    print(f'행정구역 : {reg}')
    temp_mean_list, temp_max_list, temp_min_list, rain_sum_list, sun_sum_list = [],[],[],[],[] # 중앙값 데이터 쌓을 빈 리스트 생성 (초기화)
    
    d = df[(df['행정구역'] == reg)] # 행정구역의 df 생성.
    date_list = d['일시'].unique() # 일시 리스트 생성.
    
    for date in date_list:
        r = d[(d['일시'] == date)] # 예 : 행정구역이 강원도 이면서 일시가 2000-01 인 데이터 프레임
        
        # 컬럼별 중앙값 구하기.
        temp_mean = calc_median(r, '평균기온(°C)')
        temp_max = calc_median(r, '최고기온(°C)')
        temp_min = calc_median(r, '최저기온(°C)')
        rain_sum = calc_median(r, '월합강수량(00~24h만)(mm)')
        sun_sum = calc_median(r, '합계 일사량(MJ/m2)')
        
        # 구한 중앙값들을 각 리스트에 쌓기.
        temp_mean_list.append(temp_mean)
        temp_max_list.append(temp_max)
        temp_min_list.append(temp_min)
        rain_sum_list.append(rain_sum)
        sun_sum_list.append(sun_sum)
    # 임의의 데이터 프레임으로 만들기. 
    imsi_df = pd.DataFrame({'행정구역':reg, '일시':date_list,'평균기온(°C)': temp_mean_list,
                           '최고기온(°C)': temp_max_list, '최저기온(°C)': temp_min_list,
                           '월합강수량(00~24h만)(mm)':rain_sum_list,'합계 일사량(MJ/m2)':sun_sum_list})
    # 위에서 생성한 빈 reg_df와 임의의 데이터 프레임 합치기
    reg_df = pd.concat([reg_df, imsi_df], axis=0)
reg_df.reset_index(inplace=True, drop=True)
        

  0%|          | 0/17 [00:00<?, ?it/s]

행정구역 : 강원도


  6%|▌         | 1/17 [00:25<06:49, 25.58s/it]

행정구역 : 경기도


 12%|█▏        | 2/17 [00:56<07:07, 28.51s/it]

...

 88%|████████▊ | 15/17 [06:25<00:52, 26.45s/it]

행정구역 : 제주도


 94%|█████████▍| 16/17 [06:46<00:24, 24.93s/it]

행정구역 : 세종특별자치시


100%|██████████| 17/17 [07:07<00:00, 25.17s/it]


In [31]:
df = reg_df

In [32]:
# 봄배추 3월~5월
# 고랭지배추 5월~9월
# 가을배추 7월~11월
# 겨울배추 8월~내년2월
#평균기온, 최저기온, 최고기온, 월합강수량, 합계일사량 

In [33]:
# 일시 뽑을 방법 확인
df['일시'][0][-2:]

'01'

In [34]:
# 월만 추가
# month = df['일시'].apply(lambda x:x.split('-')[-1])
# df.insert(3, '월', month)
df['월'] = df['일시'].apply(lambda x:x.split('-')[-1])
df.head()

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월
0,강원도,2000-01,-2.9,8.5,-16.0,48.1,152.990,01
1,강원도,2000-02,-3.3,7.7,-15.8,5.7,329.070,02
2,강원도,2000-03,4.4,19.3,-8.6,14.1,448.650,03
3,강원도,2000-04,10.4,23.7,-3.1,37.1,486.355,04
4,강원도,2000-05,15.9,31.6,3.9,75.2,520.280,05


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23078 entries, 0 to 23077
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                23078 non-null  object 
 1   지점명                 23078 non-null  object 
 2   일시                  23078 non-null  object 
 3   평균기온(°C)            23078 non-null  float64
 4   최고기온(°C)            23078 non-null  float64
 5   최저기온(°C)            23078 non-null  float64
 6   월합강수량(00~24h만)(mm)  23078 non-null  float64
 7   합계 일사량(MJ/m2)       23078 non-null  float64
 8   월                   23078 non-null  object 
dtypes: float64(5), object(4)
memory usage: 1.6+ MB


In [7]:
#겨울배추 추가
winter_df = df.loc[(df['월'] == '01')|(df['월'] == '02')|(df['월'] == '08')|(df['월'] == '09')|(df['월'] == '10')|(df['월'] == '11')|(df['월'] == '12')]

In [36]:
#봄배추, 고랭지배추, 가을배추 추가
spiring_df =  df.loc[(df['월'] == '03')|(df['월'] == '04')|(df['월'] == '05')]
gorange_df = df.loc[(df['월'] == '05')|(df['월'] == '06')|(df['월'] == '07')|(df['월'] == '08')|(df['월'] == '9')]
fall_df = df.loc[(df['월'] == '07')|(df['월'] == '08')|(df['월'] == '09')|(df['월'] == '10')|(df['월'] == '11')]

In [38]:
spiring_df

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월
2,강원도,2000-03,4.4,19.3,-8.6,14.1,448.650,03
3,강원도,2000-04,10.4,23.7,-3.1,37.1,486.355,04
4,강원도,2000-05,15.9,31.6,3.9,75.2,520.280,05
14,강원도,2001-03,3.3,21.6,-8.5,20.6,445.030,03
15,강원도,2001-04,12.0,27.3,-4.1,16.5,539.605,04
...,...,...,...,...,...,...,...,...
4467,세종특별자치시,2020-04,10.8,25.9,-0.8,17.6,651.275,04
4468,세종특별자치시,2020-05,18.2,29.1,7.5,91.6,643.645,05
4478,세종특별자치시,2021-03,8.8,23.3,-2.3,99.8,465.040,03
4479,세종특별자치시,2021-04,13.9,29.8,0.7,56.0,585.900,04


In [37]:
spiring_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122 entries, 2 to 4480
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                1122 non-null   object 
 1   일시                  1122 non-null   object 
 2   평균기온(°C)            1122 non-null   float64
 3   최고기온(°C)            1122 non-null   float64
 4   최저기온(°C)            1122 non-null   float64
 5   월합강수량(00~24h만)(mm)  1122 non-null   float64
 6   합계 일사량(MJ/m2)       1122 non-null   float64
 7   월                   1122 non-null   object 
dtypes: float64(5), object(3)
memory usage: 78.9+ KB


In [39]:
# 배추품종에 따라 값을 부여함

winter_df['배추품종'] = '겨울배추'
spiring_df['배추품종'] = '봄배추'
gorange_df['배추품종'] = '고랭지배추'
fall_df['배추품종'] = '겨울배추'

In [12]:
spiring_df

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),월,배추품종
2,강원도,속초,2000-03,6.9,18.5,-4.2,8.7,448.650,03,봄배추
3,강원도,속초,2000-04,11.2,24.1,2.4,23.4,486.355,04,봄배추
4,강원도,속초,2000-05,14.3,30.5,6.2,71.0,520.280,05,봄배추
14,강원도,속초,2001-03,5.6,19.9,-6.0,20.8,445.030,03,봄배추
15,강원도,속초,2001-04,12.3,25.2,1.2,14.1,539.605,04,봄배추
...,...,...,...,...,...,...,...,...,...,...
23057,경상남도,남해,2020-04,12.5,23.4,2.6,93.8,633.670,04,봄배추
23058,경상남도,남해,2020-05,18.2,27.7,9.6,184.1,614.270,05,봄배추
23068,경상남도,남해,2021-03,10.6,22.0,0.8,234.4,497.100,03,봄배추
23069,경상남도,남해,2021-04,14.3,25.0,4.5,119.7,608.970,04,봄배추


In [ ]:
# 생산량 데이터 불러와서 . . . 전처리 . . 
# pandas wide to long (검색해서 배추 생산량 데이터 ,, 데이터 프레임 형태로 만들기
# 16일은 데이터 프레임 형태 이후 합치고 전처리